<a href="https://colab.research.google.com/github/Krishan-Kumar-kk/NLP/blob/main/NLP_hack_A_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [9]:
df = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [10]:
overview=pd.read_csv("/content/game_overview.csv")

In [11]:
df.head()

,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1


In [12]:
overview.head()

,title,developer,publisher,tags,overview
0,Spooky's Jump Scare Mansion,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...
1,Sakura Clicker,Winged Cloud,Winged Cloud,"['Nudity', 'Anime', 'Free to Play', 'Mature', ...",The latest entry in the Sakura series is more ...
2,WARMODE,WARTEAM,WARTEAM,"['Early Access', 'Free to Play', 'FPS', 'Multi...",Free to play shooter about the confrontation o...
3,Fractured Space,Edge Case Games Ltd.,Edge Case Games Ltd.,"['Space', 'Multiplayer', 'Free to Play', 'PvP'...",Take the helm of a gigantic capital ship and g...
4,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,"['FPS', 'Multiplayer', 'Shooter', 'Action', 'T...",Counter-Strike: Global Offensive (CS: GO) expa...


In [13]:
test.head()

,review_id,title,year,user_review
0,1603,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B..."
1,1604,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...
2,1605,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...
3,1606,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...
4,1607,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...


In [14]:
df_Train=df.merge(overview,how='left',on='title')
print("shape of df_train:", df_Train.shape)

shape of df_train: (17494, 9)


In [15]:
df_Test=test.merge(overview,how='left',on='title')

In [16]:
df_Train.isnull().sum()

review_id            0
title                0
year               178
user_review          0
user_suggestion      0
developer            0
publisher            0
tags                 0
overview             0
dtype: int64

In [17]:
df_Train['year']=df_Train['year'].ffill(axis=0)
df_Test['year']=df_Test['year'].ffill(axis=0)

In [18]:
df_Train['year'].unique()

array([2016., 2015., 2017., 2018., 2014., 2012., 2013., 2011.])

In [19]:
X = df_Train.drop('user_suggestion',axis=1)
y=df['user_suggestion']
print(X.shape)
print(y.shape)

(17494, 8)
(17494,)


In [20]:
#Choosing vocabulary size to be 5000 and copying data to msg for further cleaning
voc_size = 5000
msg =pd.DataFrame(X['user_review'])
msg_test = test['user_review']

In [21]:
msg

,user_review
0,I'm scared and hearing creepy voices. So I'll...
1,"Best game, more better than Sam Pepper's YouTu..."
2,"A littly iffy on the controls, but once you kn..."
3,"Great game, fun and colorful and all that.A si..."
4,Not many games have the cute tag right next to...
...,...
17489,Arguably the single greatest mmorp that exists...
17490,"An older game, to be sure, but has its own cha..."
17491,When I frist started playing Everquest 2 it wa...
17492,cool game. THe only thing that REALLY PISSES M...


In [26]:
#Downloading stopwords 
#Stopwords are the words in any language which does not add much meaning to a sentence.
#They can safely be ignored without sacrificing the meaning of the sentence.
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
#We will be using Stemming here
#Stemming map words to their root forms
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [38]:
#Applying stemming and some preprocessing
for i in range(len(msg)):
  review = re.sub('[^a-zA-Z]',' ',X['user_review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)




In [43]:
#Applying stemming and some preprocessing
corpus_test=[]
for i in range(len(df_Test['user_review'])):
  review = re.sub('[^a-zA-Z]',' ',df_Test['user_review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [44]:
# Converting to one hot representation
onehot_rep = [one_hot(words,voc_size)for words in corpus]
onehot_rep_test = [one_hot(words,voc_size)for words in corpus_test]

In [45]:
#Padding Sentences to make them of same size
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

# Creating and training model

In [46]:
#We have used embedding layers with LSTM
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [47]:
#Converting into numpy array
X_final = np.array(embedded_docs)
y_final = np.array(y)
test_final = np.array(embedded_docs_test)
X_final.shape,y_final.shape,test_final.shape

((17494, 25), (17494,), (8045, 25))

In [55]:
#training model
model.fit(X_final,y_final,epochs=50,batch_size=64)

Epoch 1/50
274/274 [==============================] - 10s 36ms/step - loss: 0.0818 - accuracy: 0.9679
Epoch 2/50
274/274 [==============================] - 10s 37ms/step - loss: 0.0725 - accuracy: 0.9716
Epoch 3/50
274/274 [==============================] - 10s 37ms/step - loss: 0.0642 - accuracy: 0.9774
Epoch 4/50
274/274 [==============================] - 10s 37ms/step - loss: 0.0646 - accuracy: 0.9750
Epoch 5/50
274/274 [==============================] - 10s 37ms/step - loss: 0.0637 - accuracy: 0.9756
Epoch 6/50
274/274 [==============================] - 10s 37ms/step - loss: 0.0576 - accuracy: 0.9792
Epoch 7/50
274/274 [==============================] - 10s 36ms/step - loss: 0.0559 - accuracy: 0.9805
Epoch 8/50
274/274 [==============================] - 10s 36ms/step - loss: 0.0551 - accuracy: 0.9786
Epoch 9/50
274/274 [==============================] - 10s 37ms/step - loss: 0.0492 - accuracy: 0.9813
Epoch 10/50
274/274 [==============================] - 10s 36ms/step - loss: 0.052

In [56]:
y_pred = model.predict_classes(test_final)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [58]:
final_sub = pd.DataFrame()
final_sub['review_id']=test['review_id']
final_sub['user_suggestion'] = y_pred
final_sub.to_csv('final_sub2.csv',index=False)